In [1]:
#-*- coding: utf-8 -*-

from time import sleep
import requests
import json
from pytz import timezone
from datetime import datetime
from urllib.parse import urlencode
from bs4 import BeautifulSoup
#import urllib3
#urllib3.disable_warnings()

In [38]:
def get_tvN(day_list):
    #tvN
    #https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200501

    total = 0
    program_list = set()
        
    for craw_day in day_list:
        base_url = 'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=' + str(craw_day)
        
        arr = base_url.split('?')
        arr2 = arr[1].split('&')
        skip_word = (arr2[0].split('='))[1] + '( '
        print (base_url)
        
        req = requests.get(base_url, verify=False)
        total_data1 = req.text
        total_data2 = total_data1.replace(skip_word, '')
        total_data3 = total_data2.replace(');', '')

        var = json.loads(total_data3)
        naver_list = var['dataHtml']

        for link in naver_list:
            soup = BeautifulSoup(link, "html.parser")
            total_data = soup.find_all(attrs={'class': 'lk'})
            total = total + len(total_data)

            for link in total_data:
                program_list.add(link.text)

            #print ('\n#################################\n')
            #print (program_list)
            #print ('\n===> program_list : ', len(program_list))

    return (program_list)

In [55]:
def get_general(day_list):
    channel_list = [
        #지상파
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=100&u2=&u3=1+2+3&u5=0&u4='    
        #종편
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22408213267523362562_1588310643412&key=MultiChannelDailySchedule&where=m&pkid=66&u1=500&u2=&u3=&u5=0&u4='
        #캐이블
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=200&u2=&u3=12&u5=0&u4='
        #스카이프
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=300&u2=&u3=25&u5=0&u4='
        #라디오
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=400&u2=&u3=&u5=0&u4='
    ]

    total = 0
    program_list = set()

    for base_url in channel_list:
        
        for craw_day in day_list:
            base_url = base_url + str(craw_day)

            arr = base_url.split('?')
            arr2 = arr[1].split('&')
            skip_word = (arr2[0].split('='))[1] + '( '
            print (skip_word)

            req = requests.get(base_url, verify=False)
            total_data1 = req.text
            total_data2 = total_data1.replace(skip_word, '')
            total_data3 = total_data2.replace(');', '')

            var = json.loads(total_data3)
            naver_list = var['dataHtml']

            for link in naver_list:
                soup = BeautifulSoup(link, "html.parser")
                total_data = soup.find_all(attrs={'class': 'pr_name'})
                total = total + len(total_data)
                for p in total_data:
                    #print (p.text)
                    program_list.add(p.text)

            #print ('\n#################################\n')
            #print (program_list)
            #print ('\n===> program_list : ', len(program_list))

    return (program_list)


In [52]:
def get_pages():
    pages = set() # 애초에 set은 중복을 허용하지 않음 

    #지상파
    base_uri = 'https://m.search.naver.com/search.naver'
    base_url = 'https://m.search.naver.com/search.naver?where=m&query=%EB%B0%A9%EC%86%A1%ED%8E%B8%EC%84%B1%ED%91%9C&sm=mtb_she&qdt=0'
    req = requests.get(base_url, verify=False)
    soup = BeautifulSoup(req.text, "html.parser")
    total_data = soup.find(attrs={'class': 'channel_list'})
    
    craw_baseurl_list = set()
    for link in total_data.findAll("a"):
        if 'href' in link.attrs: 
            url = base_uri + link.attrs['href']
            print(url)
            pages.add(url)

    return (pages)

In [53]:
def craw_data(base_url):
    req = requests.get(base_url, verify=False)
    soup = BeautifulSoup(req.text, "html.parser")
    total_data = soup.find(attrs={'class': 'sc cs_tvtime _cs_tvtime'})
    total_data2 = total_data.find_all(attrs={'class': 'lk'})

    craw_index = 0
    craw_data = set()
    
    for d in total_data2:
        craw_index = craw_index + 1
        craw_data.add(d.text)
        print(craw_index, d.text)
        
    return (craw_data)

In [59]:
if __name__ == "__main__":
    fmt = "%Y-%m-%d %H:%M:%S %Z%z"
    #while (True):
    KST = datetime.now(timezone('Asia/Seoul'))
    
    program_list = set()
    
    day_list = [20200426, 20200427, 20200428, 20200429, 20200430, 20200501, 20200502, 20200503, 20200504, 20200505]
    
    p_list = get_tvN(day_list)
    for p in p_list:
        program_list.add(p)
    p_list = get_general(day_list)
    for p in p_list:
        program_list.add(p)
    
    print ('len : ', len(program_list))
    print (program_list)

https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200426
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200427
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200428


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200429
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200430
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200501


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200502
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200503
https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200504


...

jQuery22406981672323418231_1588307496613( 
jQuery22406981672323418231_1588307496613( 


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


jQuery22406981672323418231_1588307496613( 
jQuery22406981672323418231_1588307496613( 


/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


len :  622
{'도전, 골든벨', '자동공부책상 위키 2', '뉴스(뉴스투데이)', '나이트라인', '영화,왓수다', '내꺼야 폴록-숫자의 거리로 가다', '엉뚱남매 요리조리쇼', 'MBN 프레스룸', '뉴스(24:00)', 'JTBC 뉴스룸', '코로나19 통합뉴스룸 KBS 뉴스 7', '백세누리쇼', '애니갤러리', '신상출시 편스토랑', '배성재의 텐', '충격실화극 싸인', '20주년 기획 인간극장', '동상이몽2 너는 내운명', '엘리멘트리 2', 'IDOL RADIO', '딩동댕 유치원', '고스트 앤 크라임 4', '뉴스 퍼레이드', '노래의 날개 위에(재)', '오늘 아침 정지영입니다', '시사스페셜', '살맛 나는 오늘', '드래곤에그', 'NCIS:뉴올리언스 5', '세상의 모든 음악', '벅스봇', '집사부일체', '백세누리쇼 특별판', '이재익의 정치쇼 3,4부', '무한리필 샐러드', '가정의 달 특집 실화탐사대 1부', '헬스N라이프', '프리한 19', '웃고 떠들고 맛있는 하우스', '집사부일체 2부', 'KBS 네트워크 특선 고고씽', '신비한 동물퀴즈', '더 킹 : 영원의 군주 1부', '유영미의 마음은 언제나 청춘', '기적의 습관', '자이언트 펭TV 스페셜', '미래교육 플러스', '강석, 김혜영의 싱글벙글쇼', '신대동여지도', '코리언 지오그래픽 플러스', '명의', '장학퀴즈-학교에 가다', '특집다큐 장내 세균의 비밀 잘 먹고 잘 빼는 법', '생생정보 스페셜', '링크, 소프트웨어 세상', '헬로키즈 글로벌 동물 뉴스', '채널A 스페셜 - 다채로운 바다공원', '모란봉 클럽', "아메리카's 퍼니스트 홈 비디오", '헬로카봇 유니버스', '밥블레스유 2', '도올학당 수다승철', '미술 탐험대', '본격연예 한밤 2부', 'EBS 온라인개학 초등 국어 2-1 가', '식신로드 시즌4', 'TV CHOSUN 뉴스7', '스타와 직거래 유랑마켓', '진짜사랑 3', 'TV 동물농장 2부', '김이나의 밤편지